In [1]:
import os

In [2]:
pwd

'c:\\Users\\pmore4\\Desktop\\vs code\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
pwd

'c:\\Users\\pmore4\\Desktop\\vs code\\Text-Summarizer-Project'

In [18]:
!pip install protobuf

In [5]:
from  dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir :Path
    data_path : Path
    model_ckpt : Path
    num_train_epochs: 2
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
# Configuration Manager

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs= params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
# Component
from transformers import TrainingArguments,Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset,load_from_disk
import torch

c:\Users\pmore4\Desktop\vs code\Text-Summarizer-Project\textsummarize\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-08-28 12:50:56,536: INFO: config: PyTorch version 2.4.0 available.]


In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model=model_pegasus)


        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_orgs = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
            evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_steps
        )

        trainer = Trainer(model=model_pegasus,
                          args=trainer_orgs,
                          tokenizer=tokenizer,
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        # Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegausus-samsum-models"))

        #Save Tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e :
    raise e

[2024-08-28 12:50:57,064: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-28 12:50:57,069: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-28 12:50:57,071: INFO: common: Created directory at path :artifacts]
[2024-08-28 12:50:57,073: INFO: common: Created directory at path :artifacts/model_trainer]


c:\Users\pmore4\Desktop\vs code\Text-Summarizer-Project\textsummarize\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\pmore4\Desktop\vs code\Text-Summarizer-Project\textsummarize\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 

{'loss': 3.1101, 'grad_norm': 31.952730178833008, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 20%|█▉        | 20/102 [1:11:22<13:50:23, 607.60s/it]

{'loss': 3.0467, 'grad_norm': 15.775440216064453, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


...

{'loss': 2.9905, 'grad_norm': 20.176532745361328, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.78}


 49%|████▉     | 50/102 [2:24:52<2:53:18, 199.97s/it]

{'loss': 2.8545, 'grad_norm': 52.482635498046875, 'learning_rate': 5e-06, 'epoch': 0.98}


 58%|█████▊    | 59/102 [2:59:32<2:45:34, 231.04s/it]

KeyboardInterrupt: 

f